In [1]:
from sorf_query import *
import jsonlines
import pandas as pd

In [2]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 200

In [3]:
GENOME_REFERENCE_PATH = 's3://velia-annotation-dev/genomes/hg38/GRCh38.p13.genome.fa.gz'

In [4]:
with open('test_vtx.txt') as fhandle:
    ids = [int(i.replace('VTX-', '')) for i in fhandle.readlines()]
ids[:5]

[7082, 7146, 7396, 7541, 9335]

In [5]:
new_ids = [851390, 81996, 850636]
ids = new_ids+ids

In [6]:
# session.rollback()

In [7]:
session = base.Session()

In [8]:
orfs = session.query(Orf).filter(Orf.id.in_(ids)).all()
current_orf = orfs[0]
missing_orfs = set(ids) - set([i.id for i in orfs])
missing_orfs

set()

In [10]:
def compute_exact_transcript_matches(o):
    nt, aa = extract_nucleotide_sequence_broken_psl_starts(current_orf, reference)
    r = find_seq_substring(nt, transcripts)
    return o.id, nt, aa, [i.split('|')[0].split('.')[0] for i in r]

def run_id_mapping_parallel(orfs, NCPU = None):
    import multiprocessing as mp
    results = {}
    if NCPU is None:
        NCPU = mp.cpu_count()
    if NCPU > 1:
        with mp.Pool(NCPU) as ppool:            
            for r in tqdm(ppool.imap(compute_exact_transcript_matches, orfs), total=len(orfs)):
                results[r[0]] = r[1:]
    else:
        for o in tqdm(orfs):
            r = compute_exact_transcript_matches(o)
            results[r[0]] = r[1:]
    return results

In [11]:
transcript_matching_results = run_id_mapping_parallel(orfs[:8], NCPU = 1)

100%|██████████| 8/8 [00:52<00:00,  6.55s/it]


In [27]:
xena_transcripts = pd.read_csv('s3://velia-data-dev/VDC_004_annotation/20230329_tcga_data/xena_transcript_names.csv')
overlapping_xena_sorf_transcripts = set(xena_transcripts['Xena Transcripts']).intersection(set(transcripts_to_map
                                                                                            ))
overlapping_xena_sorf_transcripts

{'ENST00000298966',
 'ENST00000299415',
 'ENST00000301200',
 'ENST00000302101',
 'ENST00000319694',
 'ENST00000322329',
 'ENST00000326842',
 'ENST00000327157',
 'ENST00000525141',
 'ENST00000527149',
 'ENST00000596676'}

In [33]:
metadata = pd.read_table('s3://velia-data-dev/VDC_004_annotation/20230329_tcga_data/Xena/TcgaTargetGTEX_phenotype.txt')
metadata

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xca in position 15827: invalid continuation byte

Index(['GTEX-S4Q7-0003-SM-3NM8M', 'TCGA-19-1787-01', 'TCGA-S9-A7J2-01',
       'GTEX-QV31-1626-SM-2S1QC', 'TCGA-G3-A3CH-11',
       'GTEX-13OVI-1026-SM-5L3EM', 'GTEX-13OW5-0626-SM-5J2N2',
       'GTEX-XUZC-2026-SM-4BRW9', 'TCGA-EK-A2RE-01', 'TCGA-D5-5538-01',
       ...
       'TCGA-IB-7885-01', 'TCGA-B6-A0IA-01', 'GTEX-1117F-2826-SM-5GZXL',
       'TCGA-VQ-AA6F-01', 'TCGA-BR-8588-01', 'GTEX-11ZTS-3326-SM-5LU9Y',
       'TCGA-DD-A115-01', 'GTEX-ZA64-2126-SM-5Q5A8', 'TCGA-FV-A3I0-11',
       'GTEX-XV7Q-0426-SM-4BRVN'],
      dtype='object', length=19126)

In [39]:
    transcripts_to_load = transcript_matching_results
    
    xena_transcripts = pd.read_csv('s3://velia-data-dev/VDC_004_annotation/20230329_tcga_data/xena_transcript_names.csv')
    if transcripts_to_load is None:
        transcripts_to_map = xena_transcripts['Xena Transcripts']
    else:
        overlapping_xena_sorf_transcripts = set(xena_transcripts['Xena Transcripts']).intersection(set(transcripts_to_load))
    xena = pd.read_feather('s3://velia-data-dev/VDC_004_annotation/20230329_tcga_data/xena.feather', columns = ['index']+list(overlapping_xena_sorf_transcripts))
    xena.index = xena.pop('index')
    metadata = pd.read_table('s3://velia-data-dev/VDC_004_annotation/20230329_tcga_data/Xena/TcgaTargetGTEX_phenotype.txt',
                             encoding = "ISO-8859-1", index_col=0)
    metadata = metadata[~metadata['_primary_site'].isna()]
    ordr = xena.index.intersection(metadata.index)
    xena = xena.loc[ordr].copy()
    metadata = metadata.loc[ordr].copy()

    tissue_pairs = pd.read_excel('s3://velia-data-dev/VDC_004_annotation/20230329_tcga_data/Xena/GTEx_TCGA_comparisons.xlsx')
    tissue_pairs['GTEx Tissue Type'] = tissue_pairs['GTEx Tissue Type'].str.replace('whole ', '')
    tissue_pairs = tissue_pairs[tissue_pairs['GTEx Tissue Type']!='--']
    
    # tissue_pairs.head(4)

,TCGA Cancer Type,Description,GTEx Tissue Type,Ambiguity,Unnamed: 4,Unnamed: 5
0,ACC,Adrenocortical Cancer,adrenal gland,N,NaN,NaN
1,BLCA,bladder urothelial carcinoma,bladder,N,NaN,NaN
2,BRCA,breast invasive carcinoma,breast,N,NaN,NaN
3,CESC,Cervical & Endocervical Cancer,cervix uteri,N,NaN,NaN


In [35]:
metadata

,detailed_category,primary disease or tissue,_primary_site,_sample_type,_gender,_study
sample,,,,,,
TCGA-V4-A9EE-01,Uveal Melanoma,Uveal Melanoma,Eye,Primary Tumor,Male,TCGA
TCGA-VD-AA8N-01,Uveal Melanoma,Uveal Melanoma,Eye,Primary Tumor,Male,TCGA
TCGA-V4-A9EI-01,Uveal Melanoma,Uveal Melanoma,Eye,Primary Tumor,Male,TCGA
TCGA-VD-AA8O-01,Uveal Melanoma,Uveal Melanoma,Eye,Primary Tumor,Male,TCGA
TCGA-WC-A888-01,Uveal Melanoma,Uveal Melanoma,Eye,Primary Tumor,Male,TCGA
...,...,...,...,...,...,...
TARGET-20-PANPKN-09,Acute Myeloid Leukemia,Acute Myeloid Leukemia,White blood cell,Primary Blood Derived Cancer - Bone Marrow,Male,TARGET
TARGET-20-PANLIR-09,Acute Myeloid Leukemia,Acute Myeloid Leukemia,White blood cell,Primary Blood Derived Cancer - Bone Marrow,Male,TARGET
TARGET-20-PAPAWN-09,Acute Myeloid Leukemia,Acute Myeloid Leukemia,White blood cell,Primary Blood Derived Cancer - Bone Marrow,Male,TARGET


In [21]:
transcript_matching_results[7082][-1]

['ENST00000525141', 'ENST00000298966', 'ENST00000527149', 'ENST00000596676']

In [22]:
overlapping_tids

['XM_017023741.2', 'ENST00000329565.6']

In [25]:
transcripts_to_map = []
with jsonlines.open('sorf_table.jsonlines', mode = 'w') as fh:
    for current_orf in tqdm(orfs):
        # nt, aa = extract_nucleotide_sequence_broken_psl_starts(current_orf, reference)
        overlapping_tids = query_overlapping_transcripts(current_orf, session)
        overlapping_tids = [[i.split('.')[0] for i in [t.ensembl_id, t.refseq_id, t.chess_id] if i][0] for t in overlapping_tids]
        # exact_tids = find_seq_substring(nt, transcripts)
        # if len(tids)>0:
        # exact_tids = [i.split('|')[0].split('.')[0] for i in exact_tids]
        exact_tids = transcript_matching_results[current_orf.id][-1]
        nt = transcript_matching_results[current_orf.id][0]
        aa = transcript_matching_results[current_orf.id][1]
        attributes = {
            'chr': current_orf.assembly.ucsc_style_name,
            'vtx': f"VTX-{str(current_orf.id).zfill(7)}",
            'strand': current_orf.strand,
            'start': current_orf.start,
            'end': current_orf.end,
            'nucl': nt,
            'aa': aa,
            'transcripts_exact': exact_tids,
            'transcripts_overlapping': overlapping_tids,
            
        }
        transcripts_to_map+=exact_tids
        transcripts_to_map+=[i.split('.')[0] for i in overlapping_tids if i.startswith('ENST')]
        
        fh.write(attributes)
        # break

  1%|          | 8/644 [00:00<00:49, 12.85it/s]


KeyError: 11192

In [ ]:
assembly_ids = {}

for assembly in session.query(base.Assembly).all():
    if assembly.ucsc_style_name.startswith('chr') and len(assembly.ucsc_style_name) < 6:
        assembly_ids[assembly.ucsc_style_name] = assembly
    else:
        assembly_ids[assembly.genbank_accession] = assembly

In [ ]:
assembly_ids

In [17]:
from Bio import SeqIO

In [21]:
with smart_open.open('s3://velia-annotation-dev/gencode/v42/gencode.v42.transcripts.fa.gz') as f:
    seqs = SeqIO.to_dict(SeqIO.parse(f, 'fasta'))

In [24]:
# seqs.keys()

# Map transcripts

In [253]:
targets = {
    'VTX-0851390': 'ATGGCCGAGAGGCCGGGGCCTCCGGGCGGCGCCGTGTCCGCGACCGCGTACCCTGACACCCCCGCGGAATTCCCTCCGCACCTCCAGGCGGGTGCGATGCGGCGCCGCTTTTGGGGCGTATTCAACTGTCTGTGCGCCGGCGCGTTCGGGGCCCTGGCCGCCGCCTCCGCCAAGCTGGCCTTCGGCAGCGAGGTGAGCATGGGTTTATGCGTCTTAGGCATTATTGTGATGGCGAGCACCAATTCTCTGATGTGGACCTTCTTTAGCCGGGGCCTCAGTTTCTCCATGTCTTCAGCCATTGCATCTGTCACAGTGACTTTTTCAAATATCCTCAGCTCGGCCTTCCTGGGCTATGTGCTGTATGGAGAGTGCCAGGAGGTCTTGTGGTGGGGAGGAGTGTTCCTTATTCTCTGCGGACTCACCCTAATCCACAGGAAGCTCCCACCCACCTGGAAGCCCCTTCCACACAAGCAGCAG',
    'VTX-0081996': 'ATGGCGGATGTGTCAGAGAGGACACTGCAGTTGTCCGTGCTAGTAGCCTTCGCTTCTGGAGTACTCCTGGGCTGGCAGGCGAACCGACTGCGGAGGCGCTACTTGGACTGGAGGAAAAGGAGGCTGCAGGACAAGCTGGCGGCGACGCAGAAGAAGCTGGACCTGGCC',
    'VTX-0087927': 'ATGAACGGCTCTCAGGCGGGCGCCGCGGCTCAGGCCGCCTGGCTGAGCTCCTGCTGTAACCAGTCGGCGTCGCCGCCGGAGCCCCCCGAGGGGCCGCGCGCGGTGCAGGCGGTGGTGCTCGGCGTGCTGTCCCTGCTGGTGCTTTGCGGGGTCCTGTTCCTGGGCGGCGGCCTCCTCCTCCGCGCCCAGGGCCTGACAGCGCTGCTGACCCGCGAGCAGCGCGCGTCCCGCGAGCCCGAGCCGGGCAGTGCCAGCGGAGAGGACGGCGACGACGACTCC',
    'VTX-0085840': 'ATGCTCCTGGGGAGTCTGTGGGGAAGATGCCATCCAGGGCGCTGTGCGCTCTTCCTCATCCTCGCCCTCCTGCTGGACGCGGTCGGCCTGGTCCTTTTGCTGCTGGGGATCTTGGCCCCCCTGAGTTCCTGGGACTTCTTCATCTACACAGGTGCCCTGATCCTGGCTCTCAGCCTACTGCTCTGGATCATCTGGTATTCCCTCAACATTGAGGTGTCTCCTGAAAAACTGGACCTG',
    'VTX-0087707': 'ATGACCTCCTGGCCAGGTGGCAGCTTTGGCCCTGACCCGCTCCTGGCCCTGCTGGTGGTGATCCTGCTAGCACGCCTCATCCTGTGGTCCTGCCTCGGGACCTACATCGACTACAGACTGGCCCAGCGGCGGCCCCAGAAACCCAAGCAGGAC',
}
for i, s in targets.items():
    hit = find_seq_substring(s, transcripts)
    print(i, hit)

VTX-0851390 ['ENST00000302392.5|ENSG00000169964.8|OTTHUMG00000133092.3|OTTHUMT00000256750.3|TMEM42-201|TMEM42|977|protein_coding|']
VTX-0081996 ['ENST00000611969.5|ENSG00000175701.11|OTTHUMG00000131194.5|OTTHUMT00000477505.2|MTLN-203|MTLN|427|protein_coding|', 'ENST00000414416.2|ENSG00000175701.11|OTTHUMG00000131194.5|OTTHUMT00000253918.4|MTLN-201|MTLN|2051|protein_coding|', 'ENST00000426713.1|ENSG00000175701.11|OTTHUMG00000131194.5|OTTHUMT00000338107.3|MTLN-202|MTLN|461|protein_coding|']
VTX-0087927 ['ENST00000546390.2|ENSG00000284791.2|OTTHUMG00000169615.4|OTTHUMT00000405071.2|SMIM41-201|SMIM41|1430|protein_coding|']
VTX-0085840 ['ENST00000581851.2|ENSG00000263429.4|OTTHUMG00000132847.7|OTTHUMT00000256321.4|TMEM238L-202|TMEM238L|1535|protein_coding|']
VTX-0087707 ['ENST00000641568.1|ENSG00000284713.2|OTTHUMG00000172974.2|OTTHUMT00000491249.1|SMIM38-202|SMIM38|2695|protein_coding|', 'ENST00000686937.1|ENSG00000284713.2|OTTHUMG00000172974.2|-|SMIM38-204|SMIM38|5447|protein_coding|', 'E

In [251]:
import gzip
import fsspec
from pyfaidx import Fasta
import pandas as pd
from tqdm import tqdm

# deduped_conservation = pd.read_parquet('deduped_phylocsf_length_filtered.parq')
transcripts = Fasta('./gencode.v42.transcripts.fa')

def find_seq_substring(query, target_dict):
    return [k for k, s in target_dict.items() if query.lower() in s[:].seq.lower()]

def extract_sequence(reference, chrom, strand, block_starts, block_sizes):
    s = reference[chrom]
    sequence = ''
    for start, length in zip(block_starts, block_sizes):
        end = start + (3*(length-1)) + 3
        sequence += str(s[start : end])
    sequence+=str(s[end:end+3])
    return sequence

def pfunc(row):
    ix = row['qName']
    starts = tuple(map(int, row['tStarts'].strip(',').split(',')))
    sizes = tuple(map(int, row['blockSizes'].strip(',').split(',')))
    s = extract_sequence(genes, row['tName'], row['strand'], starts, sizes)
    # print(row['strand'], s[:3], s[-3:])
    # blat_sequences[row['qName']] = s
    matches = find_seq_substring(s, transcripts)
    if len(matches)>0:
        transcript_names = [i.split('|')[0].split('.')[0] for i in matches]
        gene_names = [i.split('|')[5] for i in matches]
        return ix, {'transcripts': transcript_names, 'genes': gene_names, 'sequence': s}
    else:
        return ix, {'transcripts': [], 'genes': [], 'sequence': s}
    
# import multiprocessing as mp
# with mp.Pool(120) as ppool:
#     annotations = {}
#     ixes, rows = list(zip(*deduped_conservation.iterrows()))
#     for r in tqdm(ppool.imap(pfunc, rows), total=50000):#, total=signalp_and_conserved.shape[0]):
#         annotations[r[0]] = r[1]